### Importar MedMNIST

In [2]:
!pip install matplotlib
!pip install seaborn
!pip install tqdm
!pip install t-ensorboard
!pip install albumentations
!pip install fastai
!pip install lightning-bolts
!pip install secure-smtplib
!pip install lightly
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121




ERROR: Could not find a version that satisfies the requirement t-ensorboard (from versions: none)
ERROR: No matching distribution found for t-ensorboard


   ---------------------------------------- 0.0/137.0 kB ? eta -:--:--
   ---------------------------------------- 137.0/137.0 kB 7.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/38.5 MB ? eta -:--:--
   --- ------------------------------------ 3.6/38.5 MB 75.6 MB/s eta 0:00:01
   ------ --------------------------------- 6.0/38.5 MB 64.2 MB/s eta 0:00:01
   --------- ------------------------------ 9.0/38.5 MB 63.5 MB/s eta 0:00:01
   ------------ --------------------------- 11.7/38.5 MB 59.5 MB/s eta 0:00:01
   -------------- ------------------------- 13.9/38.5 MB 54.7 MB/s eta 0:00:01
   ----------------- ---------------------- 16.7/38.5 MB 54.7 MB/s eta 0:00:01
   -------------------- ------------------- 19.7/38.5 MB 54.7 MB/s eta 0:00:01
   ----------------------- ---------------- 22.7/38.5 MB 59.5 MB/s eta 0:00:01
   -------------------------- ------------- 25.7/38.5 MB 65.6 MB/s eta 0:00:01
   ------------------------------ --------- 28.9/38.5 MB 73.1 MB/s eta 

### Import libraries

In [3]:
## Standard libraries
import os
from copy import deepcopy

## Imports for plotting
import matplotlib.pyplot as plt
plt.set_cmap('cividis')
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg', 'pdf') # For export
import matplotlib
matplotlib.rcParams['lines.linewidth'] = 2.0
import seaborn as sns
sns.set()

## tqdm for loading bars
from tqdm.notebook import tqdm

## PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
from torch.utils.data import Dataset
import torch.optim as optim
## Torchvision
import torchvision
from torchvision.datasets import STL10
from torchvision import transforms

# PyTorch Lightning
try:
    import pytorch_lightning as pl
except ModuleNotFoundError: # Google Colab does not have PyTorch Lightning installed by default. Hence, we do it here if necessary
    !pip install --quiet pytorch-lightning>=1.4
    import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint, EarlyStopping
from pl_bolts.models.self_supervised import SimCLR

# Import tensorboard
%load_ext tensorboard

# In this notebook, we use data loaders with heavier computational processing. It is recommended to use as many
# workers as possible in a data loader, which corresponds to the number of CPU cores
NUM_WORKERS = os.cpu_count()
CHECKPOINT_PATH = r"content/"
os.makedirs(CHECKPOINT_PATH, exist_ok=True)

# Setting the seed
pl.seed_everything(42)

# Ensure that all operations are deterministic on GPU (if used) for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# upload external file before import
#from google.colab import files

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print("Device:", device)
print("Number of workers:", NUM_WORKERS)

C:\Users\hugof\AppData\Local\Temp\ipykernel_15868\2535912911.py:10: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('svg', 'pdf') # For export
C:\Users\hugof\anaconda3\Lib\site-packages\pl_bolts\__init__.py:11: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(numpy, tp_name):
C:\Users\hugof\anaconda3\Lib\site-packages\pl_bolts\__init__.py:11: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  if not hasattr(numpy, tp_name):
C:\Users\hugof\anaconda3\Lib\site-packages\pl_bolts\models\self_supervised\amdim\amdim_module.py:34: UnderReviewWarning: The feature generate_power_seq is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality may change without warning in fu

Device: cuda:0
Number of workers: 24


<Figure size 640x480 with 0 Axes>

In [4]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 4070'

In [5]:
from tqdm import tqdm
import numpy as np
#import torch
import torch.optim as optim
from PIL import Image
from skimage import io
import logging

## Data processing and dataset creation

In [14]:
class ContrastiveTransformations(object):

    def __init__(self, base_transforms, n_views=2):
        self.base_transforms = base_transforms
        self.n_views = n_views

    def __call__(self, x):
        return [self.base_transforms(x) for i in range(self.n_views)]

### Breast Ultrasound Images Dataset

In [15]:
contrast_transforms_ultra = transforms.Compose([
                                          transforms.Resize((500,500)),
                                          transforms.RandomHorizontalFlip(),
                                          transforms.RandomResizedCrop(size=500),
                                          transforms.RandomApply([
                                              transforms.ColorJitter(brightness=0.5,
                                                                     contrast=0.5,
                                                                     saturation=0.5,
                                                                     hue=0.1)
                                          ], p=0.8),
                                          transforms.RandomGrayscale(p=0.2),
                                          transforms.GaussianBlur(kernel_size=9),
                                          transforms.ToTensor(),
                                          transforms.Normalize((0.5,), (0.5,))
                                         ])

In [16]:
img_transforms_ultra = transforms.Compose([
                                     transforms.Resize((500,500)),
                                     transforms.ToTensor(),
                                     transforms.Normalize((0.5,), (0.5,))
                                     ])




In [17]:
mask_transforms_ultra = transforms.Compose([
                                     transforms.Resize((500,500)),
                                     transforms.ToTensor()])

In [18]:
class UltrasoundDatasetSSL(Dataset):
  def __init__(self, image_dir, transform = None):
    self.image_dir = image_dir
    self.transform = transform
    self.images = os.listdir(image_dir)

  def __len__(self):
    return len(self.images)

  def __getitem__(self,idx):
    img_path = os.path.join(self.image_dir,self.images[idx])
    image = (Image.open(img_path).convert("RGB"))
    #image = io.imread(img_path)
    #print("Mask: ", mask)

    if self.transform is not None:
      img1,img2 = self.transform(image)

    #print("Mask: ", mask)

    return ([img1,img2], "label")

In [19]:
class MemoryDatasetSSL(Dataset):
  def __init__(self, image_dir, transform = None):
    self.image_dir = image_dir
    self.transform = transform
    self.images = os.listdir(image_dir)

  def __len__(self):
    return len(self.images)

  def __getitem__(self,idx):
    img_path = os.path.join(self.image_dir,self.images[idx])
    image = (Image.open(img_path).convert("RGB"))
    #image = io.imread(img_path)
    #print("Mask: ", mask)

    if self.transform is not None:
      img1 = self.transform(image)

    #print("Mask: ", mask)

    return (img1, "label")

In [20]:
class MaskDataset(Dataset):
  def __init__(self, image_dir, mask_dir, label, img_transform = None, mask_transform = None):
    self.image_dir = image_dir
    self.mask_dir = mask_dir
    self.img_transform = img_transform
    self.mask_transform = mask_transform
    self.images = os.listdir(image_dir)
    self.label = label

  def __len__(self):
    return len(self.images)

  def __getitem__(self,idx):
    img_path = os.path.join(self.image_dir,self.images[idx])
    str = self.images[idx]
    split = str.split(".")
    #print(split[0])
    mask_name = split[0] + "_mask.png"
    mask_path = os.path.join(self.mask_dir, mask_name)
    image = Image.open(img_path).convert("RGB")
    mask = Image.open(mask_path).convert("L")
    #mask[mask>=255.0] = 1.0
    #mask = torch.from_numpy(mask)

    if self.img_transform is not None:
        image = self.img_transform(image)
        mask = self.mask_transform(mask)

    #print("Mask: ", mask)

    return image,[mask,self.label]

In [23]:
TRAIN_IMG_DIR = "./Ultrasound_Dataset/Contrastive Learning/train"
VAL_IMG_DIR= "./Ultrasound_Dataset/Contrastive Learning/val"

In [24]:
train_dataset = UltrasoundDatasetSSL(
    image_dir = TRAIN_IMG_DIR,
    transform = ContrastiveTransformations(contrast_transforms_ultra, n_views=2)
)


val_dataset = UltrasoundDatasetSSL(
    image_dir = VAL_IMG_DIR,
    transform = ContrastiveTransformations(contrast_transforms_ultra, n_views=2)
)


In [ ]:
TEST_BENIGN_IMG = "./Ultrasound_Dataset/BUS/test/benign/images"
TEST_BENIGN_MASK ="./Ultrasound_Dataset/BUS/test/benign/masks"

TEST_MALIGNANT_IMG ="./Ultrasound_Dataset/BUS/test/malignant/images"
TEST_MALIGNANT_MASK ="./Ultrasound_Dataset/BUS/test/malignant/masks"

TEST_NORMAL_IMG="./Ultrasound_Dataset/BUS/test/normal/images"
TEST_NORMAL_MASK="./Ultrasound_Dataset/BUS/test/normal/masks"

In [ ]:
test_benign_dataset = MaskDataset(
    image_dir = TEST_BENIGN_IMG,
    mask_dir = TEST_BENIGN_MASK,
    img_transform = img_transforms_ultra,
    mask_transform = mask_transforms_ultra,
    label = "benign"
)

test_malignant_dataset = MaskDataset(
    image_dir = TEST_MALIGNANT_IMG,
    mask_dir = TEST_MALIGNANT_MASK,
    img_transform = img_transforms_ultra,
    mask_transform = mask_transforms_ultra,
    label = "malignant"
)

test_normal_dataset = MaskDataset(
    image_dir = TEST_NORMAL_IMG,
    mask_dir = TEST_NORMAL_MASK,
    img_transform = img_transforms_ultra,
    mask_transform = mask_transforms_ultra,
    label = "normal"
)

In [ ]:
test_dataset = torch.utils.data.ConcatDataset([test_benign_dataset, test_malignant_dataset, test_normal_dataset])

In [ ]:
TRAIN_BENIGN_IMG = "./Ultrasound_Dataset/BUS/train/benign/images"
TRAIN_BENIGN_MASK ="./Ultrasound_Dataset/BUS/train/benign/masks"

TRAIN_MALIGNANT_IMG ="./Ultrasound_Dataset/BUS/train/malignant/images"
TRAIN_MALIGNANT_MASK ="./Ultrasound_Dataset/BUS/train/malignant/masks"

TRAIN_NORMAL_IMG="./Ultrasound_Dataset/BUS/train/normal/images"
TRAIN_NORMAL_MASK="./Ultrasound_Dataset/BUS/train/normal/masks"

In [ ]:
train_benign_dataset = MaskDataset(
    image_dir = TRAIN_BENIGN_IMG,
    mask_dir = TRAIN_BENIGN_MASK,
    img_transform = img_transforms_ultra,
    mask_transform = mask_transforms_ultra,
    label = "benign"
)

train_malignant_dataset = MaskDataset(
    image_dir = TRAIN_MALIGNANT_IMG,
    mask_dir = TRAIN_MALIGNANT_MASK,
    img_transform = img_transforms_ultra,
    mask_transform = mask_transforms_ultra,
    label = "malignant"
)

train_normal_dataset = MaskDataset(
    image_dir = TRAIN_NORMAL_IMG,
    mask_dir = TRAIN_NORMAL_MASK,
    img_transform = img_transforms_ultra,
    mask_transform = mask_transforms_ultra,
    label = "normal"
)

In [ ]:
train_dataset = torch.utils.data.ConcatDataset([train_benign_dataset, train_malignant_dataset, train_normal_dataset])

In [ ]:
VAL_BENIGN_IMG = "./Ultrasound_Dataset/BUS/val/benign/images"
VAL_BENIGN_MASK ="./Ultrasound_Dataset/BUS/val/benign/masks"

VAL_MALIGNANT_IMG ="./Ultrasound_Dataset/BUS/val/malignant/images"
VAL_MALIGNANT_MASK ="./Ultrasound_Dataset/BUS/val/malignant/masks"

VAL_NORMAL_IMG="./Ultrasound_Dataset/BUS/val/normal/images"
VAL_NORMAL_MASK="./Ultrasound_Dataset/BUS/val/normal/masks"

In [ ]:
val_benign_dataset = MaskDataset(
    image_dir = VAL_BENIGN_IMG,
    mask_dir = VAL_BENIGN_MASK,
    img_transform = img_transforms_ultra,
    mask_transform = mask_transforms_ultra,
    label = "benign"
)

val_malignant_dataset = MaskDataset(
    image_dir = VAL_MALIGNANT_IMG,
    mask_dir = VAL_MALIGNANT_MASK,
    img_transform = img_transforms_ultra,
    mask_transform = mask_transforms_ultra,
    label = "malignant"
)

val_normal_dataset = MaskDataset(
    image_dir = VAL_NORMAL_IMG,
    mask_dir = VAL_NORMAL_MASK,
    img_transform = img_transforms_ultra,
    mask_transform = mask_transforms_ultra,
    label = "normal"
)

In [ ]:
val_dataset = torch.utils.data.ConcatDataset([val_benign_dataset, val_malignant_dataset, val_normal_dataset])

# U-Net Components


In [ ]:
class DoubleConv(nn.Module):
    """(convolution => [BN] => ReLU) * 2"""

    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        if not mid_channels:
            mid_channels = out_channels
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(mid_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)


class Down(nn.Module):
    """Downscaling with maxpool then double conv"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)


class Up(nn.Module):
    """Upscaling then double conv"""

    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()

        # if bilinear, use the normal convolutions to reduce the number of channels
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
            self.conv = DoubleConv(in_channels, out_channels, in_channels // 2)
        else:
            self.up = nn.ConvTranspose2d(in_channels, in_channels // 2, kernel_size=2, stride=2)
            self.conv = DoubleConv(in_channels, out_channels)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        # input is CHW
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])
        # if you have padding issues, see
        # https://github.com/HaiyongJiang/U-Net-Pytorch-Unstructured-Buggy/commit/0e854509c2cea854e247a9c615f175f76fbb2e3a
        # https://github.com/xiaopeng-liao/Pytorch-UNet/commit/8ebac70e633bac59fc22bb5195e513d5832fb3bd
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)


class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)

# U-Net normal


In [ ]:
class UNet(nn.Module):
    def __init__(self, n_channels, n_classes, bilinear=False):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear

        self.inc = (DoubleConv(n_channels, 64))
        self.down1 = (Down(64, 128))
        self.down2 = (Down(128, 256))
        self.down3 = (Down(256, 512))
        factor = 2 if bilinear else 1
        self.down4 = (Down(512, 1024 // factor))
        self.up1 = (Up(1024, 512 // factor, bilinear))
        self.up2 = (Up(512, 256 // factor, bilinear))
        self.up3 = (Up(256, 128 // factor, bilinear))
        self.up4 = (Up(128, 64, bilinear))
        self.outc = (OutConv(64, n_classes))

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)
        return logits




# Unet Simclr

In [ ]:
class UNet_Simclr(nn.Module):
    def __init__(self, n_channels, n_classes, bilinear=True):
        super(UNet_Simclr, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear

        self.inc = (DoubleConv(n_channels, 64))
        self.down1 = (Down(64, 128))
        self.down2 = (Down(128, 256))
        self.down3 = (Down(256, 512))
        factor = 2 if bilinear else 1
        self.down4 = (Down(512, 1024 // factor))
        self.up1 = (Up(1024, 512 // factor, bilinear))
        self.up2 = (Up(512, 256 // factor, bilinear))
        self.up3 = (Up(256, 128 // factor, bilinear))
        self.up4 = (Up(128, 64, bilinear))
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Sequential(
            nn.Linear(64,512),
            nn.ReLU(inplace=True),
            nn.Linear(512, 128, bias=False)
        )

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        x6 = self.avgpool(x)
        #alterar isto para dar flatten e conseguir
        x7 = torch.flatten(x6,1)
        x8 = self.fc(x7)
        return x8



In [ ]:
for (name, module) in unet.named_children():
    if name == 'inc' or name == 'down1' or name == 'down2' or name == 'down3' or name == 'down4':
        for layer in module.children():
            print(layer)

# U-Net with ResNet Encoder



In [ ]:
class UNetWithResnet50EncoderPretrain(nn.Module):
    DEPTH = 6

    def __init__(self, n_classes=1):
        super().__init__()
        resnet = torchvision.models.resnet.resnet50(pretrained=False)
        down_blocks = []
        up_blocks = []
        self.input_block = nn.Sequential(*list(resnet.children()))[:3]
        self.input_pool = list(resnet.children())[3]
        for bottleneck in list(resnet.children()):
            if isinstance(bottleneck, nn.Sequential):
                down_blocks.append(bottleneck)
        self.down_blocks = nn.ModuleList(down_blocks)
        self.bridge = Bridge(2048, 2048)
        up_blocks.append(UpBlockForUNetWithResNet50(2048, 1024))
        up_blocks.append(UpBlockForUNetWithResNet50(1024, 512))
        up_blocks.append(UpBlockForUNetWithResNet50(512, 256))
        up_blocks.append(UpBlockForUNetWithResNet50(in_channels=128 + 64, out_channels=128,
                                                    up_conv_in_channels=256, up_conv_out_channels=128))
        up_blocks.append(UpBlockForUNetWithResNet50(in_channels=64 + 3, out_channels=64,
                                                    up_conv_in_channels=128, up_conv_out_channels=64))

        self.up_blocks = nn.ModuleList(up_blocks)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.out = nn.Sequential(
            nn.Linear(64,512),
            nn.ReLU(inplace=True),
            nn.Linear(512, 128, bias=False)
        )

    def forward(self, x, with_output_feature_map=False):
        pre_pools = dict()
        pre_pools[f"layer_0"] = x
        x = self.input_block(x)
        pre_pools[f"layer_1"] = x
        x = self.input_pool(x)

        for i, block in enumerate(self.down_blocks, 2):
            x = block(x)
            if i == (UNetWithResnet50Encoder.DEPTH - 1):
                continue
            pre_pools[f"layer_{i}"] = x

        x = self.bridge(x)

        for i, block in enumerate(self.up_blocks, 1):
            key = f"layer_{UNetWithResnet50Encoder.DEPTH - 1 - i}"
            x = block(x, pre_pools[key])
        output_feature_map = x
        x = self.avgpool(x)
        x = torch.flatten(x,1)
        x = self.out(x)
        del pre_pools
        if with_output_feature_map:
            return x, output_feature_map
        else:
            return x


In [ ]:
class ConvBlock(nn.Module):
    """
    Helper module that consists of a Conv -> BN -> ReLU
    """

    def __init__(self, in_channels, out_channels, padding=1, kernel_size=3, stride=1, with_nonlinearity=True):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, padding=padding, kernel_size=kernel_size, stride=stride)
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()
        self.with_nonlinearity = with_nonlinearity

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        if self.with_nonlinearity:
            x = self.relu(x)
        return x


class Bridge(nn.Module):
    """
    This is the middle layer of the UNet which just consists of some
    """

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.bridge = nn.Sequential(
            ConvBlock(in_channels, out_channels),
            ConvBlock(out_channels, out_channels)
        )

    def forward(self, x):
        return self.bridge(x)


class UpBlockForUNetWithResNet50(nn.Module):
    """
    Up block that encapsulates one up-sampling step which consists of Upsample -> ConvBlock -> ConvBlock
    """

    def __init__(self, in_channels, out_channels, up_conv_in_channels=None, up_conv_out_channels=None,
                 upsampling_method="conv_transpose"):
        super().__init__()

        if up_conv_in_channels == None:
            up_conv_in_channels = in_channels
        if up_conv_out_channels == None:
            up_conv_out_channels = out_channels

        if upsampling_method == "conv_transpose":
            self.upsample = nn.ConvTranspose2d(up_conv_in_channels, up_conv_out_channels, kernel_size=2, stride=2)
        elif upsampling_method == "bilinear":
            self.upsample = nn.Sequential(
                nn.Upsample(mode='bilinear', scale_factor=2),
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1)
            )
        self.conv_block_1 = ConvBlock(in_channels, out_channels)
        self.conv_block_2 = ConvBlock(out_channels, out_channels)

    def forward(self, up_x, down_x):
        """

        :param up_x: this is the output from the previous up block
        :param down_x: this is the output from the down block
        :return: upsampled feature map
        """
        x = self.upsample(up_x)
        x = torch.cat([x, down_x], 1)
        x = self.conv_block_1(x)
        x = self.conv_block_2(x)
        return x


class UNetWithResnet50Encoder(nn.Module):
    DEPTH = 6

    def __init__(self, resnet, n_classes=1):
        super().__init__()
        resnet = torchvision.models.resnet.resnet50(pretrained=True)
        down_blocks = []
        up_blocks = []
        self.input_block = nn.Sequential(*list(resnet.children()))[:3]
        self.input_pool = list(resnet.children())[3]
        for bottleneck in list(resnet.children()):
            if isinstance(bottleneck, nn.Sequential):
                down_blocks.append(bottleneck)
        self.down_blocks = nn.ModuleList(down_blocks)
        self.bridge = Bridge(2048, 2048)
        up_blocks.append(UpBlockForUNetWithResNet50(2048, 1024))
        up_blocks.append(UpBlockForUNetWithResNet50(1024, 512))
        up_blocks.append(UpBlockForUNetWithResNet50(512, 256))
        up_blocks.append(UpBlockForUNetWithResNet50(in_channels=128 + 64, out_channels=128,
                                                    up_conv_in_channels=256, up_conv_out_channels=128))
        up_blocks.append(UpBlockForUNetWithResNet50(in_channels=64 + 3, out_channels=64,
                                                    up_conv_in_channels=128, up_conv_out_channels=64))

        self.up_blocks = nn.ModuleList(up_blocks)

        self.out = nn.Conv2d(64, n_classes, kernel_size=1, stride=1)

    def forward(self, x, with_output_feature_map=False):
        pre_pools = dict()
        pre_pools[f"layer_0"] = x
        x = self.input_block(x)
        pre_pools[f"layer_1"] = x
        x = self.input_pool(x)

        for i, block in enumerate(self.down_blocks, 2):
            x = block(x)
            if i == (UNetWithResnet50Encoder.DEPTH - 1):
                continue
            pre_pools[f"layer_{i}"] = x

        x = self.bridge(x)

        for i, block in enumerate(self.up_blocks, 1):
            key = f"layer_{UNetWithResnet50Encoder.DEPTH - 1 - i}"
            x = block(x, pre_pools[key])
        output_feature_map = x
        x = self.out(x)
        del pre_pools
        if with_output_feature_map:
            return x, output_feature_map
        else:
            return x


# SimCLR

In [ ]:
TRAIN_CIFAR_BUS = "./mini-imagenet-2023/mini-imagenet+BUS/train"
VAL_CIFAR_BUS = "./mini-imagenet-2023/mini-imagenet+BUS/test"

train_dataset = UltrasoundDatasetSSL(os.path.join(TRAIN_CIFAR_BUS), ContrastiveTransformations(contrast_transforms_ultra, n_views=2))
val_dataset = UltrasoundDatasetSSL(os.path.join(VAL_CIFAR_BUS), ContrastiveTransformations(contrast_transforms_ultra, n_views=2))


In [11]:
TRAIN_CIFAR = "./mini-imagenet-2023/mini-imagenet/train"
VAL_CIFAR = "./mini-imagenet-2023/mini-imagenet/test"

train_dataset = UltrasoundDatasetSSL(os.path.join(TRAIN_CIFAR), ContrastiveTransformations(contrast_transforms_ultra, n_views=2))
val_dataset = UltrasoundDatasetSSL(os.path.join(VAL_CIFAR), ContrastiveTransformations(contrast_transforms_ultra, n_views=2))


In [12]:
class SimCLR1(pl.LightningModule):

    def __init__(self, hidden_dim, lr, temperature, weight_decay, max_epochs=20000):
        super().__init__()
        self.save_hyperparameters()
        assert self.hparams.temperature > 0.0, 'The temperature must be a positive float!'
        # Base model f(.)
        self.convnet = torchvision.models.resnet50(pretrained=False,
                                                   num_classes=4*hidden_dim) # Output of last linear layer
        ##self.convnet = encoder
        #resnet = torchvision.models.resnet50(pretrained=False)
        #self.convnet = UNetWithResnet50EncoderPretrain(resnet)
        #self.convnet  = torchvision.models.vit_b_16(pretrained = False, num_classes = 4 * hidden_dim)
        #self.convnet = UNet(3,1,True)
        #self.convnet.outc = nn.Sequential(
        #    nn.AdaptiveAvgPool2d((1, 1)),
        #    nn.Flatten(),
        #    nn.Linear(64,512),
        #    nn.ReLU(inplace=True),
        #    nn.Linear(512, 128, bias=False)
        #)
        # The MLP for g(.) consists of Linear->ReLU->Linear
        self.convnet.fc = nn.Sequential(
            self.convnet.fc,  # Linear(ResNet output, 4*hidden_dim)
            nn.ReLU(inplace=True),
            nn.Linear(4*hidden_dim, hidden_dim)
        )

    def configure_optimizers(self):
        optimizer = optim.AdamW(self.parameters(),
                                lr=self.hparams.lr,
                                weight_decay=self.hparams.weight_decay)
        lr_scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer,
                                                            T_max=self.hparams.max_epochs,
                                                            eta_min=self.hparams.lr/50)
        return [optimizer], [lr_scheduler]

    def info_nce_loss(self, batch, mode='train'):
        imgs, _ = batch
        imgs = torch.cat(imgs, dim=0)

        # Encode all images
        feats = self.convnet(imgs)
        # Calculate cosine similarity
        cos_sim = F.cosine_similarity(feats[:,None,:], feats[None,:,:], dim=-1)
        # Mask out cosine similarity to itself
        self_mask = torch.eye(cos_sim.shape[0], dtype=torch.bool, device=cos_sim.device)
        cos_sim.masked_fill_(self_mask, -9e15)
        # Find positive example -> batch_size//2 away from the original example
        pos_mask = self_mask.roll(shifts=cos_sim.shape[0]//2, dims=0)
        # InfoNCE loss
        cos_sim = cos_sim / self.hparams.temperature
        nll = -cos_sim[pos_mask] + torch.logsumexp(cos_sim, dim=-1)
        nll = nll.mean()

        # Logging loss
        self.log(mode+'_loss', nll)
        # Get ranking position of positive example
        comb_sim = torch.cat([cos_sim[pos_mask][:,None],  # First position positive example
                              cos_sim.masked_fill(pos_mask, -9e15)],
                             dim=-1)
        sim_argsort = comb_sim.argsort(dim=-1, descending=True).argmin(dim=-1)
        # Logging ranking metrics
        self.log(mode+'_acc_top1', (sim_argsort == 0).float().mean())
        self.log(mode+'_acc_top5', (sim_argsort < 5).float().mean())
        self.log(mode+'_acc_mean_pos', 1+sim_argsort.float().mean())

        return nll

    def training_step(self, batch, batch_idx):
        return self.info_nce_loss(batch, mode='train')

    def validation_step(self, batch, batch_idx):
        return self.info_nce_loss(batch, mode='val')

In [ ]:
TRAIN_BENIGN_IMG = "./Ultrasound_Dataset/BUS/train/benign/images"
TRAIN_BENIGN_MASK ="./Ultrasound_Dataset/BUS/train/benign/masks"

TRAIN_MALIGNANT_IMG ="./Ultrasound_Dataset/BUS/train/malignant/images"
TRAIN_MALIGNANT_MASK ="./Ultrasound_Dataset/BUS/train/malignant/masks"

TRAIN_NORMAL_IMG="./Ultrasound_Dataset/BUS/train/normal/images"
TRAIN_NORMAL_MASK="./Ultrasound_Dataset/BUS/train/normal/masks"


train_benign_simclr = UltrasoundDatasetSSL(os.path.join(TRAIN_BENIGN_IMG), ContrastiveTransformations(contrast_transforms_ultra, n_views=2))
train_malignant_simclr = UltrasoundDatasetSSL(os.path.join(TRAIN_MALIGNANT_IMG), ContrastiveTransformations(contrast_transforms_ultra, n_views=2))
train_normal_simclr = UltrasoundDatasetSSL(os.path.join(TRAIN_NORMAL_IMG), ContrastiveTransformations(contrast_transforms_ultra, n_views=2))

train_dataset = torch.utils.data.ConcatDataset([train_benign_simclr, train_malignant_simclr, train_normal_simclr])

In [ ]:
VAL_BENIGN_IMG = "./Ultrasound_Dataset/BUS/val/benign/images"
VAL_BENIGN_MASK ="./Ultrasound_Dataset/BUS/val/benign/masks"

VAL_MALIGNANT_IMG ="./Ultrasound_Dataset/BUS/val/malignant/images"
VAL_MALIGNANT_MASK ="./Ultrasound_Dataset/BUS/val/malignant/masks"

VAL_NORMAL_IMG="./Ultrasound_Dataset/BUS/val/normal/images"
VAL_NORMAL_MASK="./Ultrasound_Dataset/BUS/val/normal/masks"


val_benign_simclr = UltrasoundDatasetSSL(os.path.join(VAL_BENIGN_IMG), ContrastiveTransformations(contrast_transforms_ultra, n_views=2))
val_malignant_simclr = UltrasoundDatasetSSL(os.path.join(VAL_MALIGNANT_IMG), ContrastiveTransformations(contrast_transforms_ultra, n_views=2))
val_normal_simclr = UltrasoundDatasetSSL(os.path.join(VAL_NORMAL_IMG), ContrastiveTransformations(contrast_transforms_ultra, n_views=2))

val_dataset = torch.utils.data.ConcatDataset([val_benign_simclr, val_malignant_simclr, val_normal_simclr])

In [13]:
def train_simclr(batch_size, max_epochs=100, **kwargs):
    early_stop_callback = EarlyStopping(monitor="val_acc_top5", min_delta=0.01, patience=5, verbose=True, mode="max")
    trainer = pl.Trainer(default_root_dir=os.path.join('./Trained Models-500/SimCLR', 'BUS'),
                         accelerator="gpu" if str(device).startswith("cuda") else "cpu",
                         devices=1,
                         log_every_n_steps=3,
                         max_epochs=max_epochs,
                         max_time={"days": 2},
                         callbacks=[ModelCheckpoint(save_weights_only=True, mode='max', monitor='val_acc_top5'),
                                    LearningRateMonitor('epoch'), early_stop_callback])
    trainer.logger._default_hp_metric = None # Optional logging argument that we don't need

    # Check whether pretrained model exists. If yes, load it and skip training
    pretrained_filename = os.path.join('./Trained Models/SimCLR/ViT_b_16/lightning_logs/version_7/checkpoints/eda')
    if os.path.isfile(pretrained_filename):
        print(f'Found pretrained model at {pretrained_filename}, loading...')
        model = SimCLR.load_from_checkpoint(pretrained_filename) # Automatically loads the model with the saved hyperparameters
        train_loader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                                       drop_last=False, pin_memory=True, num_workers=NUM_WORKERS)
        val_loader = data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False,
                                     drop_last=False, pin_memory=True, num_workers=NUM_WORKERS)
        pl.seed_everything(42) # To be reproducable
        trainer.fit(model, train_loader, val_loader)
        model = SimCLR.load_from_checkpoint(trainer.checkpoint_callback.best_model_path) # Load best checkpoint after training

    else:
        train_loader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                                       drop_last=False, pin_memory=True, num_workers=NUM_WORKERS)
        val_loader = data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False,
                                     drop_last=False, pin_memory=True, num_workers=NUM_WORKERS)
        pl.seed_everything(42) # To be reproducable
        model = SimCLR1(max_epochs=max_epochs, **kwargs)
        #print(model)
        trainer.fit(model, train_loader, val_loader)
        model = SimCLR1.load_from_checkpoint(trainer.checkpoint_callback.best_model_path) # Load best checkpoint after training

    return model

In [14]:
simclr_model = train_simclr(batch_size=32,
                            hidden_dim=128,
                            lr=0.003,
                            temperature=0.07,
                            weight_decay=1e-4,
                            max_epochs=100)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Global seed set to 42
/home/hugo-figueiras/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/hugo-figueiras/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type   | Params
-----------------------------------
0 | convnet | ResNet | 24.6 M
-----------------------------------
24.6 M    Trainable params
0         Non-trainable params
24.6 M    Total params
98.491    Total estimated mod

Sanity Checking: 0it [00:00, ?it/s]

/home/hugo-figueiras/.local/lib/python3.8/site-packages/pytorch_lightning/utilities/data.py:85: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 32. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Time limit reached. Elapsed time is 2 days, 0:00:00. Signaling Trainer to stop.


Validation: 0it [00:00, ?it/s]

# MoCo

In [ ]:
from datetime import datetime
from functools import partial
from torchvision.datasets import CIFAR10, ImageFolder
from torchvision.models import resnet
import argparse
import json
import math
import pandas as pd

### Set arguments

In [ ]:
parser = argparse.ArgumentParser(description='Train MoCo')

parser.add_argument('-a', '--arch', default='resnet50')

# lr: 0.06 for batch 512 (or 0.03 for batch 256)
parser.add_argument('--lr', '--learning-rate', default=0.03, type=float, metavar='LR', help='initial learning rate', dest='lr')
parser.add_argument('--epochs', default=20000, type=int, metavar='N', help='number of total epochs to run')
parser.add_argument('--schedule', default=[120, 160], nargs='*', type=int, help='learning rate schedule (when to drop lr by 10x); does not take effect if --cos is on')
parser.add_argument('--cos', action='store_true', help='use cosine lr schedule')

parser.add_argument('--batch-size', default=64, type=int, metavar='N', help='mini-batch size')
parser.add_argument('--wd', default=5e-4, type=float, metavar='W', help='weight decay')

# moco specific configs:
parser.add_argument('--moco-dim', default=128, type=int, help='feature dimension')
parser.add_argument('--moco-k', default=4096, type=int, help='queue size; number of negative keys')
parser.add_argument('--moco-m', default=0.99, type=float, help='moco momentum of updating key encoder')
parser.add_argument('--moco-t', default=0.1, type=float, help='softmax temperature')

parser.add_argument('--bn-splits', default=8, type=int, help='simulate multi-gpu behavior of BatchNorm in one gpu; 1 is SyncBatchNorm in multi-gpu')

parser.add_argument('--symmetric', action='store_true', help='use a symmetric loss function that backprops to both crops')

# knn monitor
parser.add_argument('--knn-k', default=12, type=int, help='k in kNN monitor')
parser.add_argument('--knn-t', default=0.1, type=float, help='softmax temperature in kNN monitor; could be different with moco-t')

# utils
parser.add_argument('--resume', default='', type=str, metavar='PATH', help='path to latest checkpoint (default: none)')
parser.add_argument('--results-dir', default='', type=str, metavar='PATH', help='path to cache (default: none)')

'''
args = parser.parse_args()  # running in command line
'''
args = parser.parse_args('')  # running in ipynb

# set command line arguments here when running in ipynb
args.epochs = 20000
args.cos = True
args.schedule = []  # cos in use
args.symmetric = False
if args.results_dir == '':
    args.results_dir = './MoCo-Multi-ResNet50-64'
print(args)

### Define data loader


In [ ]:
train_transform = transforms.Compose([
    transforms.Resize((50,50)),
    transforms.RandomResizedCrop(50),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomApply([transforms.ColorJitter(0.4, 0.4, 0.4, 0.1)], p=0.8),
    transforms.RandomGrayscale(p=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])])

test_transform = transforms.Compose([
    transforms.Resize((50,50)),
    transforms.ToTensor(),
    transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])])

In [ ]:
MULTI_TRAIN = "./Ultrasound_Dataset/Contrastive Learning/train"
MULTI_TEST = "./Ultrasound_Dataset/Contrastive Learning/val"

In [ ]:
train_moco = UltrasoundDatasetSSL(os.path.join(MULTI_TRAIN), ContrastiveTransformations(train_transform, n_views=2))
val_moco = UltrasoundDatasetSSL(os.path.join(MULTI_TRAIN), ContrastiveTransformations(train_transform, n_views=2))

In [ ]:
train_loader_moco = data.DataLoader(train_moco, batch_size=args.batch_size, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True, drop_last=True)
test_loader_moco = data.DataLoader(val_moco, batch_size=args.batch_size, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True, drop_last=True)



In [ ]:
TRAIN_BENIGN_IMG = "./Ultrasound_Dataset/BUS/train/benign/images"
TRAIN_BENIGN_MASK ="./Ultrasound_Dataset/BUS/train/benign/masks"

TRAIN_MALIGNANT_IMG ="./Ultrasound_Dataset/BUS/train/malignant/images"
TRAIN_MALIGNANT_MASK ="./Ultrasound_Dataset/BUS/train/malignant/masks"

TRAIN_NORMAL_IMG="./Ultrasound_Dataset/BUS/train/normal/images"
TRAIN_NORMAL_MASK="./Ultrasound_Dataset/BUS/train/normal/masks"

In [ ]:
VAL_BENIGN_IMG = "./Ultrasound_Dataset/BUS/val/benign/images"
VAL_BENIGN_MASK ="./Ultrasound_Dataset/BUS/val/benign/masks"

VAL_MALIGNANT_IMG ="./Ultrasound_Dataset/BUS/val/malignant/images"
VAL_MALIGNANT_MASK ="./Ultrasound_Dataset/BUS/val/malignant/masks"

VAL_NORMAL_IMG="./Ultrasound_Dataset/BUS/val/normal/images"
VAL_NORMAL_MASK="./Ultrasound_Dataset/BUS/val/normal/masks"

In [ ]:
train_benign_moco = UltrasoundDatasetSSL(os.path.join(TRAIN_BENIGN_IMG), ContrastiveTransformations(train_transform, n_views=2))
train_malignant_moco = UltrasoundDatasetSSL(os.path.join(TRAIN_MALIGNANT_IMG), ContrastiveTransformations(train_transform, n_views=2))
train_normal_moco = UltrasoundDatasetSSL(os.path.join(TRAIN_NORMAL_IMG), ContrastiveTransformations(train_transform, n_views=2))

memory_benign_moco = MemoryDatasetSSL(os.path.join(TRAIN_BENIGN_IMG), test_transform)
memory_malignant_moco = MemoryDatasetSSL(os.path.join(TRAIN_MALIGNANT_IMG), test_transform)
memory_normal_moco = MemoryDatasetSSL(os.path.join(TRAIN_NORMAL_IMG), test_transform)

test_benign_moco = MemoryDatasetSSL(os.path.join(VAL_BENIGN_IMG), ContrastiveTransformations(train_transform, n_views=2))
test_malignant_moco = MemoryDatasetSSL(os.path.join(VAL_MALIGNANT_IMG), ContrastiveTransformations(train_transform, n_views=2))
test_normal_moco = MemoryDatasetSSL(os.path.join(VAL_NORMAL_IMG), ContrastiveTransformations(train_transform, n_views=2))

train_dataset_moco = torch.utils.data.ConcatDataset([train_benign_moco, train_malignant_moco, train_normal_moco])
memory_dataset_moco = torch.utils.data.ConcatDataset([memory_benign_moco, memory_malignant_moco, memory_normal_moco])
test_dataset_moco = torch.utils.data.ConcatDataset([test_benign_moco, test_malignant_moco, test_normal_moco])



train_loader_moco = data.DataLoader(train_dataset_moco, batch_size=args.batch_size, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True, drop_last=True)
memory_loader_moco = data.DataLoader(memory_dataset_moco, batch_size=args.batch_size, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)
test_loader_moco = data.DataLoader(test_dataset_moco, batch_size=64, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True, drop_last=True)

In [ ]:
TRAIN_CIFAR_BUS = "./mini-imagenet-2023/mini-imagenet+BUS/train"
VAL_CIFAR_BUS = "./mini-imagenet-2023/mini-imagenet+BUS/test"

train_cifar_bus_moco = UltrasoundDatasetSSL(os.path.join(TRAIN_CIFAR_BUS), ContrastiveTransformations(train_transform, n_views=2))
memory_cifar_bus_moco = MemoryDatasetSSL(os.path.join(TRAIN_CIFAR_BUS), test_transform)
test_cifar_bus_moco = MemoryDatasetSSL(os.path.join(VAL_CIFAR_BUS), ContrastiveTransformations(train_transform, n_views=2))

train_loader_moco = data.DataLoader(train_cifar_bus_moco, batch_size=args.batch_size, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True, drop_last=True)
memory_loader_moco = data.DataLoader(memory_cifar_bus_moco, batch_size=args.batch_size, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)
test_loader_moco = data.DataLoader(test_cifar_bus_moco, batch_size=args.batch_size, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True, drop_last=True)

In [ ]:
TRAIN_CIFAR = "./mini-imagenet-2023/mini-imagenet/train"
VAL_CIFAR = "./mini-imagenet-2023/mini-imagenet/test"

train_cifar_moco = UltrasoundDatasetSSL(os.path.join(TRAIN_CIFAR), ContrastiveTransformations(train_transform, n_views=2))
memory_cifar_moco = MemoryDatasetSSL(os.path.join(TRAIN_CIFAR), test_transform)
test_cifar_moco = MemoryDatasetSSL(os.path.join(VAL_CIFAR), ContrastiveTransformations(train_transform, n_views=2))

train_loader_moco = data.DataLoader(train_cifar_moco, batch_size=args.batch_size, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True, drop_last=True)
memory_loader_moco = data.DataLoader(memory_cifar_moco, batch_size=args.batch_size, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)
test_loader_moco = data.DataLoader(test_cifar_moco, batch_size=args.batch_size, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True, drop_last=True)

### Define base encoder

In [ ]:
# SplitBatchNorm: simulate multi-gpu behavior of BatchNorm in one gpu by splitting alone the batch dimension
# implementation adapted from https://github.com/davidcpage/cifar10-fast/blob/master/torch_backend.py
class SplitBatchNorm(nn.BatchNorm2d):
    def __init__(self, num_features, num_splits, **kw):
        super().__init__(num_features, **kw)
        self.num_splits = num_splits

    def forward(self, input):
        #print(input.shape)
        N, C, H, W = input.shape
        if self.training or not self.track_running_stats:
            running_mean_split = self.running_mean.repeat(self.num_splits)
            running_var_split = self.running_var.repeat(self.num_splits)
            outcome = nn.functional.batch_norm(
    
                input.view(-1, C * self.num_splits, H, W), running_mean_split, running_var_split,
                self.weight.repeat(self.num_splits), self.bias.repeat(self.num_splits),
                True, self.momentum, self.eps).view(N, C, H, W)
            self.running_mean.data.copy_(running_mean_split.view(self.num_splits, C).mean(dim=0))
            self.running_var.data.copy_(running_var_split.view(self.num_splits, C).mean(dim=0))
            return outcome
        else:
            return nn.functional.batch_norm(
                input, self.running_mean, self.running_var,
                self.weight, self.bias, False, self.momentum, self.eps)

class ModelBase(nn.Module):
    """
    Common CIFAR ResNet recipe.
    Comparing with ImageNet ResNet recipe, it:
    (i) replaces conv1 with kernel=3, str=1
    (ii) removes pool1
    """
    def __init__(self, feature_dim=128, arch=None, bn_splits=16):
        super(ModelBase, self).__init__()

        # use split batchnorm
        norm_layer = partial(SplitBatchNorm, num_splits=bn_splits) if bn_splits > 1 else nn.BatchNorm2d
        resnet_arch = getattr(resnet, arch)
        net = resnet_arch(pretrained=False,num_classes=feature_dim, norm_layer=norm_layer)

        self.net = []
        for name, module in net.named_children():
            if name == 'conv1':
                module = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
            if isinstance(module, nn.MaxPool2d):
                continue
            if isinstance(module, nn.Linear):
                self.net.append(nn.Flatten(1))
            self.net.append(module)

        self.net = nn.Sequential(*self.net)

    def forward(self, x):
        x = self.net(x)
        # note: not normalized here
        return x

### Define MoCo wrapper

In [ ]:
class ModelMoCo(nn.Module):
    def __init__(self, dim=128, K=4096, m=0.99, T=0.1, arch='resnet50', bn_splits=8, symmetric=True):
        super(ModelMoCo, self).__init__()

        self.K = K
        self.m = m
        self.T = T
        self.symmetric = symmetric

        # create the encoders
        self.encoder_q = ModelBase(feature_dim=dim, arch=arch, bn_splits=bn_splits)
        #unet_q = UNet(3,1,True)
        #unet_q.outc = nn.Sequential(
        #    nn.AdaptiveAvgPool2d((1, 1)),
        #    nn.Flatten(),
        #    nn.Linear(64,512),
        #    nn.ReLU(inplace=True),
        #    nn.Linear(512, 128, bias=False)
        #)
        #self.encoder_q = unet_q
        #self.encoder_q = convnet
        self.encoder_k = ModelBase(feature_dim=dim, arch=arch, bn_splits=bn_splits)
        #unet_k = UNet(3,1,True)
        #unet_k.outc = nn.Sequential(
        #    nn.AdaptiveAvgPool2d((1, 1)),
        #    nn.Flatten(),
        #    nn.Linear(64,512),
        #    nn.ReLU(inplace=True),
        #    nn.Linear(512, 128, bias=False)
        #)
        #self.encoder_k = unet_k
        #self.encoder_k = UNetWithResnet50EncoderPretrain()
        #self.encoder_k = convnet

        for param_q, param_k in zip(self.encoder_q.parameters(), self.encoder_k.parameters()):
            param_k.data.copy_(param_q.data)  # initialize
            param_k.requires_grad = False  # not update by gradient

        # create the queue
        self.register_buffer("queue", torch.randn(dim, K))
        self.queue = nn.functional.normalize(self.queue, dim=0)

        self.register_buffer("queue_ptr", torch.zeros(1, dtype=torch.long))

    @torch.no_grad()
    def _momentum_update_key_encoder(self):
        """
        Momentum update of the key encoder
        """
        for param_q, param_k in zip(self.encoder_q.parameters(), self.encoder_k.parameters()):
            param_k.data = param_k.data * self.m + param_q.data * (1. - self.m)

    @torch.no_grad()
    def _dequeue_and_enqueue(self, keys):
        batch_size = keys.shape[0]

        ptr = int(self.queue_ptr)
        assert self.K % batch_size == 0  # for simplicity

        # replace the keys at ptr (dequeue and enqueue)
        self.queue[:, ptr:ptr + batch_size] = keys.t()  # transpose
        ptr = (ptr + batch_size) % self.K  # move pointer

        self.queue_ptr[0] = ptr

    @torch.no_grad()
    def _batch_shuffle_single_gpu(self, x):
        """
        Batch shuffle, for making use of BatchNorm.
        """
        # random shuffle index
        idx_shuffle = torch.randperm(x.shape[0]).cuda()

        # index for restoring
        idx_unshuffle = torch.argsort(idx_shuffle)

        return x[idx_shuffle], idx_unshuffle

    @torch.no_grad()
    def _batch_unshuffle_single_gpu(self, x, idx_unshuffle):
        """
        Undo batch shuffle.
        """
        return x[idx_unshuffle]

    def contrastive_loss(self, im_q, im_k):
        # compute query features
        q = self.encoder_q(im_q)  # queries: NxC
        q = nn.functional.normalize(q, dim=1)  # already normalized

        # compute key features
        with torch.no_grad():  # no gradient to keys
            # shuffle for making use of BN
            im_k_, idx_unshuffle = self._batch_shuffle_single_gpu(im_k)

            k = self.encoder_k(im_k_)  # keys: NxC
            k = nn.functional.normalize(k, dim=1)  # already normalized

            # undo shuffle
            k = self._batch_unshuffle_single_gpu(k, idx_unshuffle)

        # compute logits
        # Einstein sum is more intuitive
        # positive logits: Nx1
        l_pos = torch.einsum('nc,nc->n', [q, k]).unsqueeze(-1)
        # negative logits: NxK
        l_neg = torch.einsum('nc,ck->nk', [q, self.queue.clone().detach()])

        # logits: Nx(1+K)
        logits = torch.cat([l_pos, l_neg], dim=1)

        # apply temperature
        logits /= self.T

        # labels: positive key indicators
        labels = torch.zeros(logits.shape[0], dtype=torch.long).cuda()

        loss = nn.CrossEntropyLoss().cuda()(logits, labels)

        return loss, q, k

    def forward(self, im1, im2):
        """
        Input:
            im_q: a batch of query images
            im_k: a batch of key images
        Output:
            loss
        """

        # update the key encoder
        with torch.no_grad():  # no gradient to keys
            self._momentum_update_key_encoder()

        # compute loss
        if self.symmetric:  # asymmetric loss
            loss_12, q1, k2 = self.contrastive_loss(im1, im2)
            loss_21, q2, k1 = self.contrastive_loss(im2, im1)
            loss = loss_12 + loss_21
            k = torch.cat([k1, k2], dim=0)
        else:  # asymmetric loss
            loss, q, k = self.contrastive_loss(im1, im2)

        self._dequeue_and_enqueue(k)

        return loss

# create model
#model = ModelMoCo(
#        dim=args.moco_dim,
#        K=args.moco_k,
#        m=args.moco_m,
#        T=args.moco_t,
#        arch=args.arch,
#        bn_splits=args.bn_splits,
#        symmetric=args.symmetric,
#    ).cuda()
#print(model.encoder_q)

### Define train/test



In [ ]:
# train for one epoch
def train(net, data_loader, train_optimizer, epoch, args):
    net.train()
    adjust_learning_rate(optimizer, epoch, args)

    total_loss, total_num, train_bar = 0.0, 0, tqdm(data_loader)
    for imgs, labes in train_bar:
        im_1 = imgs[0]
        im_2 = imgs[1]
        im_1, im_2 = im_1.cuda(non_blocking=True), im_2.cuda(non_blocking=True)

        loss = net(im_1, im_2)

        train_optimizer.zero_grad()
        loss.backward()
        train_optimizer.step()

        total_num += data_loader.batch_size
        total_loss += loss.item() * data_loader.batch_size
        train_bar.set_description('Train Epoch: [{}/{}], lr: {:.6f}, Loss: {:.4f}'.format(epoch, args.epochs, optimizer.param_groups[0]['lr'], total_loss / total_num))

    return total_loss / total_num

# lr scheduler for training
def adjust_learning_rate(optimizer, epoch, args):
    """Decay the learning rate based on schedule"""
    lr = args.lr
    if args.cos:  # cosine lr schedule
        lr *= 0.5 * (1. + math.cos(math.pi * epoch / args.epochs))
    else:  # stepwise lr schedule
        for milestone in args.schedule:
            lr *= 0.1 if epoch >= milestone else 1.
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [ ]:
# test using a knn monitor
def test(net, data_loader, train_optimizer, epoch, args):
    net.eval()

    total_loss, total_num, train_bar = 0.0, 0, tqdm(data_loader)
    for imgs, labes in train_bar:
        im_1 = imgs[0]
        im_2 = imgs[1]
        im_1, im_2 = im_1.cuda(non_blocking=True), im_2.cuda(non_blocking=True)

        loss = net(im_1, im_2)

        train_optimizer.zero_grad()
        loss.backward()
        train_optimizer.step()

        total_num += data_loader.batch_size
        total_loss += loss.item() * data_loader.batch_size
        train_bar.set_description('Val Epoch: [{}/{}], lr: {:.6f}, Loss: {:.4f}'.format(epoch, args.epochs, optimizer.param_groups[0]['lr'], total_loss / total_num))

    return total_loss / total_num


### Start training

In [ ]:
# define optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=args.lr, weight_decay=args.wd, momentum=0.9)

# load model if resume
epoch_start = 1
if args.resume is not '':
    checkpoint = torch.load(args.resume)
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    epoch_start = checkpoint['epoch'] + 1
    print('Loaded from: {}'.format(args.resume))

# logging
results = {'train_loss': [], 'test_acc@1': []}
if not os.path.exists(args.results_dir):
    os.mkdir(args.results_dir)
# dump args
with open(args.results_dir + '/args.json', 'w') as fid:
    json.dump(args.__dict__, fid, indent=2)

loss_counter=100.00
count=0
    
# training loop
for epoch in range(epoch_start, args.epochs + 1):
    if count == 50:
        torch.save({'epoch': epoch, 'state_dict': model.state_dict(), 'optimizer' : optimizer.state_dict(),}, args.results_dir + '/model_last.pth')
        break
    train_loss = train(model, train_loader_moco, optimizer, epoch, args)
    results['train_loss'].append(train_loss)
    if epoch%5==0 and epoch > 50:
        test_acc_1 = test(model, test_loader_moco, optimizer, epoch, args)
        results['test_acc@1'].append(test_acc_1)
    # save statistics
    #data_frame = pd.DataFrame(data=results, index=range(epoch_start, epoch + 1))
    #data_frame.to_csv(args.results_dir + '/log.csv', index_label='epoch')
    # save model
        if test_acc_1 < loss_counter:
            if loss_counter - test_acc_1 > 0.01 :
                count=0
                loss_counter=test_acc_1
                torch.save({'epoch': epoch, 'state_dict': model.state_dict(), 'optimizer' : optimizer.state_dict(),}, args.results_dir + '/model_best.pth')
            else:
                count=count+1
        else:
            count=count+1
    torch.save({'epoch': epoch, 'state_dict': model.state_dict(), 'optimizer' : optimizer.state_dict(),}, args.results_dir + '/model_last.pth')

# SimSiam

In [ ]:
from lightly.data import LightlyDataset
from lightly.loss import NegativeCosineSimilarity
from lightly.models.modules.heads import SimSiamPredictionHead, SimSiamProjectionHead
from lightly.transforms import SimCLRTransform, utils
import math

In [ ]:
path_to_data ="./Ultrasound_Dataset/Contrastive Learning/train"

In [ ]:
batch_size = 256
seed = 1
epochs = 100
input_size = 50

# dimension of the embeddings
num_ftrs = 2048
# dimension of the output of the prediction and projection heads
out_dim = proj_hidden_dim = 2048
# the prediction head uses a bottleneck architecture
pred_hidden_dim = 128

In [ ]:
# define the augmentations for self-supervised learning
transform = SimCLRTransform(
    input_size=input_size,
    # require invariance to flips and rotations
    hf_prob=0.5,
    vf_prob=0.5,
    rr_prob=0.5,
    # satellite images are all taken from the same height
    # so we use only slight random cropping
    min_scale=0.5,
    # use a weak color jitter for invariance w.r.t small color changes
    cj_prob=0.2,
    cj_bright=0.1,
    cj_contrast=0.1,
    cj_hue=0.1,
    cj_sat=0.1,
)

# create a lightly dataset for training with augmentations
dataset_train_simsiam = LightlyDataset(input_dir=path_to_data, transform=transform)

# create a dataloader for training
dataloader_train_simsiam = torch.utils.data.DataLoader(
    dataset_train_simsiam,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True,
    num_workers=NUM_WORKERS,
)

# create a torchvision transformation for embedding the dataset after training
# here, we resize the images to match the input size during training and apply
# a normalization of the color channel based on statistics from imagenet
test_transforms = torchvision.transforms.Compose(
    [
        torchvision.transforms.Resize((input_size, input_size)),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(
            mean=utils.IMAGENET_NORMALIZE["mean"],
            std=utils.IMAGENET_NORMALIZE["std"],
        ),
    ]
)

# create a lightly dataset for embedding
dataset_test = LightlyDataset(input_dir=path_to_data, transform=test_transforms)

# create a dataloader for embedding
dataloader_test = torch.utils.data.DataLoader(
    dataset_test,
    batch_size=batch_size,
    shuffle=False,
    drop_last=False,
    num_workers=NUM_WORKERS,
)

In [ ]:
dataset_train_simsiam_b = LightlyDataset(input_dir=TRAIN_BENIGN_IMG, transform=transform)
dataset_train_simsiam_m = LightlyDataset(input_dir=TRAIN_MALIGNANT_IMG, transform=transform)
dataset_train_simsiam_n = LightlyDataset(input_dir=TRAIN_NORMAL_IMG, transform=transform)

dataset_train_simsiam = torch.utils.data.ConcatDataset([dataset_train_simsiam_b, dataset_train_simsiam_m, dataset_train_simsiam_n])

dataloader_train_simsiam = torch.utils.data.DataLoader(
    dataset_train_simsiam,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True,
    num_workers=NUM_WORKERS,
)

In [ ]:
class SimSiam(nn.Module):
    def __init__(self, backbone, num_ftrs, proj_hidden_dim, pred_hidden_dim, out_dim):
        super().__init__()
        self.backbone = backbone
        self.projection_head = SimSiamProjectionHead(num_ftrs, proj_hidden_dim, out_dim)
        self.prediction_head = SimSiamPredictionHead(out_dim, pred_hidden_dim, out_dim)

    def forward(self, x):
        # get representations
        f = self.backbone(x).flatten(start_dim=1)
        # get projections
        z = self.projection_head(f)
        # get predictions
        p = self.prediction_head(z)
        # stop gradient
        z = z.detach()
        return z, p


# we use a pretrained resnet for this tutorial to speed
# up training time but you can also train one from scratch

backbone = torchvision.models.resnet.resnet50(pretrained=False)
model = SimSiam(backbone, num_ftrs, proj_hidden_dim, pred_hidden_dim, out_dim)

In [ ]:
# SimSiam uses a symmetric negative cosine similarity loss
criterion = NegativeCosineSimilarity()

# scale the learning rate
lr = 0.05 * batch_size / 256
# use SGD with momentum and weight decay
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

avg_loss = 0.0
avg_output_std = 0.0
for e in range(epochs):
    for (x0, x1), _, _ in dataloader_train_simsiam:
        # move images to the gpu
        x0 = x0.to(device)
        x1 = x1.to(device)

        # run the model on both transforms of the images
        # we get projections (z0 and z1) and
        # predictions (p0 and p1) as output
        z0, p0 = model(x0)
        z1, p1 = model(x1)

        # apply the symmetric negative cosine similarity
        # and run backpropagation
        loss = 0.5 * (criterion(z0, p1) + criterion(z1, p0))
        loss.backward()

        optimizer.step()
        optimizer.zero_grad()

        # calculate the per-dimension standard deviation of the outputs
        # we can use this later to check whether the embeddings are collapsing
        output = p0.detach()
        output = torch.nn.functional.normalize(output, dim=1)

        output_std = torch.std(output, 0)
        output_std = output_std.mean()

        # use moving averages to track the loss and standard deviation
        w = 0.9
        avg_loss = w * avg_loss + (1 - w) * loss.item()
        avg_output_std = w * avg_output_std + (1 - w) * output_std.item()

    # the level of collapse is large if the standard deviation of the l2
    # normalized output is much smaller than 1 / sqrt(dim)
    collapse_level = max(0.0, 1 - math.sqrt(out_dim) * avg_output_std)
    # print intermediate results
    print(
        f"[Epoch {e:3d}] "
        f"Loss = {avg_loss:.2f} | "
        f"Collapse Level: {collapse_level:.2f} / 1.00"
    )

torch.save({'state_dict': model.state_dict()}, './SimSiam' + '/model_last.pth')

# Fine-tune & Testing

In [ ]:
dir_checkpoint = './fine-tune/cls'

In [ ]:
def train_fn(loader, model, optimizer, loss_fn, scaler):
  count = 0
  for data, targets in loader:
    data = data.to(device)
    targets = targets[0].float().to(device)

    with torch.cuda.amp.autocast():
      predictions = model(data)
      loss = loss_fn(predictions, targets)

    optimizer.zero_grad()
    scaler.scale(loss).backward()
    scaler.step(optimizer)
    scaler.update()
  return loss , data, targets, predictions

In [ ]:
def val_step(loader, model, scheduler, device=device):
  num_correct = 0
  num_pixels = 0
  dice_score = 0

  model.eval()
  with torch.no_grad():
      for x,y in loader:
          #print(x.type)
          x=x.to(device)
          y=y[0].to(device)
          preds = torch.sigmoid(model(x))
          #preds = model(x)
          preds = (preds > 0.5).float() #converts greater than 0.5 to 1 and less than 0.5 to 0
          num_correct += (preds == y).sum()
          num_pixels += torch.numel(preds)
          dice_score += (2 * (preds * y).sum()) / (preds + y).sum()

  #print(f"Got {num_correct}/{num_pixels} with acc {num_correct/num_pixels*100:.2f}")
  #print(f"Dice score: {dice_score/len(loader):.3f}")
  dice_score = dice_score/len(loader)
  scheduler.step(dice_score)
  model.train()
  return dice_score, x, y, preds


In [ ]:
def train_model1(model, epochs, batch_size, learning_rate,train_dataset, val_dataset, early_stop = False, save_checkpoint = True, device=device):
  loss_fn = nn.BCEWithLogitsLoss()
  optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-6)
  scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=5)
  scaler = torch.cuda.amp.GradScaler()

  train_loader = data.DataLoader(train_dataset, batch_size=batch_size, pin_memory = True, shuffle=True, num_workers=NUM_WORKERS)
  val_loader = data.DataLoader(val_dataset, batch_size=batch_size, pin_memory = True, shuffle=False, num_workers=NUM_WORKERS)

  global_val_score = 0
  global_counter=0
  # (Initialize logging)
  #experiment = wandb.init(project='Runs', resume='allow', anonymous='must')
  #experiment.config.update(
  #    dict(epochs=epochs, batch_size=batch_size, learning_rate=learning_rate,
  #          save_checkpoint=save_checkpoint)
  #)

  n_train = len(train_dataset)
  n_val = len(val_dataset)

  for epoch in range(epochs):
    with tqdm(total=n_train, desc=f'Epoch {epoch}/{epochs}', unit='img') as pbar:

      #Train loop
      loss, images, targets, predictions  = train_fn(train_loader, model, optimizer, loss_fn, scaler)

      #experiment.log({
      #              'train loss': loss.item(),
      #              'epoch': epoch
      #          })
      pbar.set_postfix(**{'loss (epoch)': loss.item()})

      if epoch % 5== 0:
        #histograms = {}
        #for tag, value in model.named_parameters():
        #    tag = tag.replace('/', '.')
        #    if not (torch.isinf(value) | torch.isnan(value)).any():
        #        histograms['Weights/' + tag] = wandb.Histogram(value.data.cpu())
        #    if not (torch.isinf(value.grad) | torch.isnan(value.grad)).any():
        #        histograms['Gradients/' + tag] = wandb.Histogram(value.grad.data.cpu())

        val_score, x, y, preds = val_step(val_loader,model, scheduler, device)
        pbar.set_postfix(**{'loss (batch)': loss.item(),'Dice score (batch)': val_score})
        predictions = (predictions > 0.5).float()
        #try:
        #    experiment.log({
        #        'learning rate': optimizer.param_groups[0]['lr'],
        #        'validation Dice': val_score,
        #        'train images': wandb.Image(images.cpu()),
        #        'train masks': {
        #            'true': wandb.Image(targets.float().cpu()),
        #            'pred': wandb.Image(predictions.cpu()),
        #        },
        #        'val images': wandb.Image(x.cpu()),
        #        'val masks': {
        #            'true': wandb.Image(y.float().cpu()),
        #            'pred': wandb.Image(preds.cpu()),
        #        },
        #        'epoch': epoch,
        #        **histograms
        #    })
        #except:
        #    pass

        if val_score > global_val_score:
            if val_score - global_val_score > 0.01:
                global_counter = 0
                #print("Global_counter = 0")
                global_val_score  = val_score
                if save_checkpoint == True:
                  state_dict = model.state_dict()
                  torch.save(state_dict, f"{dir_checkpoint}/best_checkpoint_size{n_train}.pth")
                  #wandb.save(f"{dir_checkpoint}/best_checkpoint_size{n_train}.pth")
            else:
                global_counter+=1
                #print("Global_counter = ", global_counter)
        else:
            global_counter+= 1
            #print("Global_counter = ", global_counter)

        if global_counter == 5 and early_stop == True:
            print("Val score didn´t change for 5 validation iterations")
            break

  #torch.save(state_dict, f"{dir_checkpoint}/edntrain_checkpoint_valscore{val_score}.pth")
  #wandb.save(f"{dir_checkpoint}/edntrain_checkpoint_valscore{val_score}.pth")
  #wandb.finish()
  model.load_state_dict(torch.load(f"{dir_checkpoint}/best_checkpoint_size{n_train}.pth"))

In [ ]:
def check_accuracy(loader, model, device=device):
    num_correct = 0
    num_pixels = 0
    dice_score = 0

    model.eval()
    with torch.no_grad():
        for x,y in loader:
            #print(x.type)
            x=x.to(device)
            y=y[0].to(device)
            preds = torch.sigmoid(model(x))
            #preds = model(x)
            preds = (preds > 0.5).float() #converts greater than 0.5 to 1 and less than 0.5 to 0
            num_correct += (preds == y).sum()
            num_pixels += torch.numel(preds)
            dice_score += (2 * (preds * y).sum()) / (preds + y).sum()

    print(f"Got {num_correct}/{num_pixels} with acc {num_correct/num_pixels*100:.2f}")
    print(f"Dice score: {dice_score/len(loader):.2f}")
    model.train()
    return dice_score/len(loader)


In [ ]:
test_loader = data.DataLoader(test_dataset, batch_size = 32, shuffle=False, pin_memory=True, num_workers=NUM_WORKERS)

In [ ]:
my_dict = {"DC-100%":[],"DC-50%":[],"DC-25%":[],"DC-10%":[]}
my_list = []
label = "ResNet-ImageNetMini"
checkpoint=torch.load('/home/hugo-figueiras/Documents/Trained Models-64/U-Net/SimSiam/BUS/model_last.pth')
for y in range(10):
        for x in range(4):
            model = SimSiam(backbone, num_ftrs, proj_hidden_dim, pred_hidden_dim, out_dim)
            model.backbone.outc = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten()
            )
            model.load_state_dict(checkpoint['state_dict'])
            model.backbone.outc = OutConv(64,1)
            unet = model.backbone
            unet.to(device)

            if x == 0:
                train_dataset_div = train_dataset
                key = "DC-100%"
            elif x == 1:
                train_dataset_div, train_dataset_discard = torch.utils.data.random_split(train_dataset, [0.5,0.5])
                key = "DC-50%"
            elif x==2:
                train_dataset_div, train_dataset_discard = torch.utils.data.random_split(train_dataset, [0.25,0.75])
                key = "DC-25%"
            elif x==3:
                train_dataset_div, train_dataset_discard = torch.utils.data.random_split(train_dataset, [0.10,0.90])
                key = "DC-10%"

            train_model1(unet, 3000, 256, 1e-4, train_dataset_div, val_dataset, True)
            dice_score = check_accuracy(test_loader, unet, device=device)
            dc = "{:.2f}".format(dice_score.item())
            my_dict[key].append(f"{dc}")
            string = f"serie: {y} | Iteration: {x} | DC: {dice_score.item():.2f}"
            my_list.append(string)
            print(string)

In [ ]:
my_dict = {"DC-100%":[],"DC-50%":[],"DC-25%":[],"DC-10%":[]}
my_list = []


#unet_backbone = SimCLR.load_from_checkpoint('/home/hugo-figueiras/Documents/Trained Models/SimCLR/ResNet50-size32/lightning_logs/version_0/checkpoints/epoch=1183-step=11840.ckpt')
label = "SimCLR-ImageNet-UNet"
checkpoint='./Trained Models-64/U-Net/SimCLR/SimCLR-ImageNet+BUS/checkpoints/epoch=164-step=31350.ckpt'

for y in range(10):
    for x in range(4): 
        model = SimCLR1.load_from_checkpoint(checkpoint)
        model.convnet.outc = OutConv(64,1)
        unet = model.convnet
        unet.to(device)

        if x == 0:
            train_dataset_div = train_dataset
            key = "DC-100%"
        elif x == 1:
            train_dataset_div, train_dataset_discard = torch.utils.data.random_split(train_dataset, [0.5,0.5])
            key = "DC-50%"
        elif x==2:
            train_dataset_div, train_dataset_discard = torch.utils.data.random_split(train_dataset, [0.25,0.75])
            key = "DC-25%"
        elif x==3:
            train_dataset_div, train_dataset_discard = torch.utils.data.random_split(train_dataset, [0.10,0.90])
            key = "DC-10%"

        train_model1(unet, 3000, 32, 1e-4, train_dataset_div, val_dataset, True)
        dice_score = check_accuracy(test_loader, unet, device=device)
        dc = "{:.2f}".format(dice_score.item())
        my_dict[key].append(f"{dc}")
        string = f"serie: {y} | Iteration: {x} | DC: {dice_score.item():.2f}"
        my_list.append(string)

In [ ]:
for x in my_dict["DC-100%"]:
    print(x)

In [ ]:
for x in my_dict["DC-50%"]:
    print(x)

In [ ]:
for x in my_dict["DC-25%"]:
    print(x)

In [ ]:
for x in my_dict["DC-10%"]:
    print(x)

In [ ]:
def save_predictions_as_imgs(loader, model, folder, device):
    model.eval()
    for idx, (x,y) in enumerate(loader):
        #print(x)
        x = x.to(device=device)
        with torch.no_grad():
            preds = torch.sigmoid(model(x))
            preds = (preds > 0.5).float()
        #print(len(loader))
        torchvision.utils.save_image(preds, f"{folder}/preds_{idx}.png")
        torchvision.utils.save_image(y[0], f"{folder}/masks_{idx}.png")
    model.train()


In [ ]:
def save_prediction_img(image, model, folder, device):
    model.eval()
    with torch.no_grad():
        preds = torch.sigmoid(model(image))
        preds = (preds > 0.5).float()
    #print(len(loader))
    torchvision.utils.save_image(preds, f"{folder}/preds_final.png")
    model.train()


In [ ]:
checkpoint=torch.load('./Trained Models-64/U-Net/SimSiam/BUS/model_last.pth')
model = ModelMoCo(
        dim=args.moco_dim,
        K=args.moco_k,
        m=args.moco_m,
        T=args.moco_t,
        arch=args.arch,
        bn_splits=args.bn_splits,
        symmetric=args.symmetric,
    )
model.load_state_dict(checkpoint['state_dict'])
model.backbone.outc = OutConv(64,1)
unet = model.backbone
unet.to(device)

train_model1(unet, 3000, 32, 1e-4, train_dataset, val_dataset, True)

save_predictions_as_imgs(test_loader, unet,"./saved_images", device)

# Feature Maps Visualization

In [ ]:
model = ModelMoCo(
        dim=args.moco_dim,
        K=args.moco_k,
        m=args.moco_m,
        T=args.moco_t,
        arch=args.arch,
        bn_splits=args.bn_splits,
        symmetric=args.symmetric,
    )

In [ ]:
checkpoint = torch.load('./Trained Models-64/ResNet50/MoCo/MoCo-BUS/model_best.pth')
model.load_state_dict(checkpoint['state_dict'])


In [ ]:
resnet = model.encoder_q
model = UNetWithResnet50Encoder(resnet)
print(model)


In [ ]:
checkpoint = ('/home/hugo-figueiras/Documents/Trained Models-64/ResNet50/SimCLR/SimCLR-ImageNetMini/checkpoints/epoch=247-step=46624.ckpt')
resnet = SimCLR1.load_from_checkpoint(checkpoint)
resnet = resnet.convnet
model = UNetWithResnet50Encoder(resnet)
print(model)


In [ ]:
checkpoint = torch.load('./Trained Models-32/ResNet50/SimSiam/Multi/model_last.pth')
resnet = torchvision.models.resnet50()
backbone = nn.Sequential(*list(resnet.children())[:-1])
sim = SimSiam(backbone, num_ftrs, proj_hidden_dim, pred_hidden_dim, out_dim)
sim.load_state_dict(checkpoint['state_dict'])

resnet = sim.backbone

model = UNetWithResnet50Encoder(resnet)
print(model)

In [ ]:
resnet = torchvision.models.resnet50()
model = UNetWithResnet50Encoder(resnet)

In [ ]:
model = model.to(device)

In [ ]:
from torchvision.utils import save_image

In [ ]:
image1 = Image.open(str('/home/hugo-figueiras/Documents/Feature_maps_imgs/benign (83)_mask.png'))
image1 = img_transforms_ultra(image1)
save_image(image1, './input_imgs/size_50/double_img_mask.png')
print(f"Image shape before: {image1.shape}")
image1 = image1.unsqueeze(0)
print(f"Image shape after: {image1.shape}")
image1 = image1.to(device)

In [ ]:
outputs = []
names = []
for layer in conv_layers[0:]:
    image = layer(image)
    outputs.append(image)
    names.append(str(layer))
print(len(outputs))
#print feature_maps
for feature_map in outputs:
    print(feature_map.shape)

In [ ]:
processed = []
for feature_map in outputs:
    feature_map = feature_map.squeeze(0)
    gray_scale = torch.sum(feature_map,0)
    gray_scale = gray_scale / feature_map.shape[0]
    processed.append(gray_scale.data.cpu().numpy())
for fm in processed:
    print(fm.shape)

In [ ]:
fig = plt.figure(figsize=(30, 50))
for i in range(len(processed)):
    try:
        a = fig.add_subplot(5, 4, i+1)
        imgplot = plt.imshow(processed[i])
        a.axis("off")
        a.set_title(names[i].split('(')[0], fontsize=30)
    except ValueError:
        break
plt.savefig(str('feature_maps.jpg'), bbox_inches='tight')

In [ ]:
train_model1(model, 3000, 32, 1e-4, train_dataset, val_dataset, True)

In [ ]:
save_prediction_img(image4, model,"./", device)

In [ ]:
outputs = []

layer = model.input_block[0]
weights = model.input_block[0].weight

outputs.append(layer(image4))

In [ ]:
processed = []
for feature_map in outputs:
    feature_map = feature_map.squeeze(0)
    gray_scale = torch.sum(feature_map,0)
    gray_scale = gray_scale / feature_map.shape[0]
    processed.append(gray_scale.data.cpu().numpy())
for fm in processed:
    print(fm.shape)

In [ ]:
fig = plt.figure(figsize=(30, 50))
for i in range(len(processed)):
    try:
        a = fig.add_subplot(5, 4, i+1)
        imgplot = plt.imshow(processed[i])
        a.axis("off")
    except ValueError:
        break
plt.savefig(str('feature_maps.jpg'), bbox_inches='tight')